In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googlqeapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

4664
Target: 0, (not a real disaster)
Text:
@bentossell @ProductHunt Thanks! I know you all get inundated. Have a good one!

---
Target: 1, (real disaster)
Text:
Suicide bomber kills 15 in Saudi security site mosque - A suicide bomber killed at least 15 people in an attack on... http://t.co/FY0r9o7Xsl

---
Target: 0, (not a real disaster)
Text:
What you gonna do now puppies?! No more destroying my #iPhone Lightning cables! https://t.co/Z4jyHaRreW

---
Target: 0, (not a real disaster)
Text:
Rescue of the day: ItÛªs World Cat Day Saturday Aug 8th Black Black/White Cats just $5 http://t.co/oqb7DaSMVy #NewBeginningsAnimalRescue

---
Target: 1, (real disaster)
Text:
LLF TALK WORLD NEWS U.S. in record hurricane drought - The United States hasn't been hit by a major hurricane in t... http://t.co/oqeq4ueGF8

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
4 police officers arrested for abusing children at police-run boot camp in San Luis Obispo Calif. - @ABC7: http://t.co/QpWOtugUI9      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 178,   77, 1802, 1043,   10, 6279,  656,   17, 4892, 3143,  906,
           4, 1483, 3661, 5010]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Ready for my close up... Errrr nope!! #notgoingoutinthat #hailstorm #alberta @HellOnWheelsAMC @HoW_fans @TalkingHell http://t.co/9gIAXD6JTY

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04530041, -0.03718175, -0.04787734, ...,  0.03177131,
          0.01306747, -0.0456303 ],
        [-0.03980108, -0.00265955,  0.01477677, ..., -0.00315977,
          0.04630092, -0.00706939],
        [-0.01670755,  0.04070539, -0.03500526, ..., -0.04064985,
         -0.0115184 , -0.00689869],
        ...,
        [ 0.01137346, -0.02861202,  0.02830474, ..., -0.03788811,
          0.03953104, -0.02667246],
        [-0.00179715, -0.01624858,  0.00894459, ..., -0.0068845 ,
          0.01142391, -0.01618128],
        [ 0.02988626,  0.02892124, -0.02413566, ..., -0.03052253,
          0.04260964, -0.00118957]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.04530041, -0.03718175, -0.04787734, -0.01985284, -0.01174311,
       -0.0439656 ,  0.01763396,  0.01101271,  0.03258898, -0.01892912,
       -0.00822784,  0.01952906, -0.02057079,  0.00248396, -0.03751434,
        0.01955361,  0.03215617, -0.0239815 ,  0.04430691, -0.02124251,
       -0.02729571, -0.04228146,  0.03003364,  0.02528096,  0.02786819,
        0.03309036,  0.02258262,  0.00889023, -0.02649533,  0.04059071,
       -0.01184011, -0.01214081, -0.00948421,  0.03163209, -0.00110202,
       -0.00673227, -0.01401932, -0.01439838,  0.00687858, -0.0447518 ,
       -0.00522581, -0.00060268, -0.00066794, -0.00179669,  0.0276759 ,
       -0.0255358 ,  0.03299541, -0.03814336, -0.03528609,  0.03291195,
       -0.04451732,  0.02400451,  0.0496111 ,  0.03007443,  0.02950222,
       -0.02703453, -0.0311581 , -0.03090551, -0.00940969, -0.04996152,
        0.01821226, -0.03301375, -0.00714928, -0.02808573, -0.00023274,
        0.030247

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230817-003433
Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6099 - accuracy: 0.6925 - val_loss: 0.5363 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4410 - accuracy: 0.8184 - val_loss: 0.4697 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3460 - accuracy: 0.8618 - val_loss: 0.4598 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2845 - accuracy: 0.8929 - val_loss: 0.4651 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2376 - accuracy: 0.9121 - val_loss: 0.4777 - val_accuracy: 0.7861


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4777 - accuracy: 0.7861


[0.47767698764801025, 0.7860892415046692]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)
(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230817-003442
Epoch 1/5
215/215 [==============================] - 7s 22ms/step - loss: 0.5075 - accuracy: 0.7500 - val_loss: 0.4574 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3269 - accuracy: 0.8686 - val_loss: 0.5041 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2283 - accuracy: 0.9145 - val_loss: 0.5552 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1682 - accuracy: 0.9394 - val_loss: 0.6153 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1156 - accuracy: 0.9561 - val_loss: 0.9079 - val_accuracy: 0.7612


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[2.7288026e-03],
       [6.5842372e-01],
       [9.9943787e-01],
       [5.4361731e-02],
       [7.3983939e-04],
       [9.9955779e-01],
       [9.2246741e-01],
       [9.9963605e-01],
       [9.9958986e-01],
       [1.0099122e-01]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.11548556430446,
 'precision': 0.7639062094795714,
 'recall': 0.7611548556430446,
 'f1': 0.758270611394164}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.2328 - accuracy: 0.9073 - val_loss: 0.5303 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1606 - accuracy: 0.9416 - val_loss: 0.5950 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1291 - accuracy: 0.9534 - val_loss: 0.7357 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1102 - accuracy: 0.9599 - val_loss: 0.7519 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0925 - accuracy: 0.9664 - val_loss: 0.7313 - val_accuracy: 0.7887


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[6.1086845e-02],
       [8.2203412e-01],
       [9.9946803e-01],
       [4.9206026e-02],
       [9.7508647e-04],
       [9.9211991e-01],
       [7.4264663e-01],
       [9.9968922e-01],
       [9.9940062e-01],
       [6.5311617e-01]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 78.87139107611549,
 'precision': 0.7909484312997928,
 'recall': 0.7887139107611548,
 'f1': 0.7867378765635805}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 8s 22ms/step - loss: 0.5063 - accuracy: 0.7489 - val_loss: 0.4577 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3112 - accuracy: 0.8734 - val_loss: 0.5214 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2052 - accuracy: 0.9219 - val_loss: 0.5620 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1387 - accuracy: 0.9539 - val_loss: 0.6657 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1010 - accuracy: 0.9673 - val_loss: 0.7606 - val_accuracy: 0.7585


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.5097 - accuracy: 0.7453 - val_loss: 0.4582 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3122 - accuracy: 0.8742 - val_loss: 0.5194 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2106 - accuracy: 0.9180 - val_loss: 0.5567 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1444 - accuracy: 0.9520 - val_loss: 0.6651 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1005 - accuracy: 0.9648 - val_loss: 0.7450 - val_accuracy: 0.7546


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.07420877],
       [0.9186039 ],
       [0.99963546],
       [0.3463921 ],
       [0.00563478],
       [0.99840456],
       [0.97668844],
       [0.9998517 ],
       [0.9998735 ],
       [0.3323221 ]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 75.45931758530183,
 'precision': 0.7549177216974436,
 'recall': 0.7545931758530183,
 'f1': 0.754722914278016}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03054602,  0.00565677,  0.01073353, ...,  0.01410936,
          0.01715625, -0.00251443],
        [-0.00534654, -0.03359807, -0.04551395, ..., -0.035409  ,
         -0.05758198, -0.0357897 ],
        [ 0.00521703, -0.04901723,  0.01754445, ..., -0.01807433,
         -0.0152657 , -0.01019085],
        ...,
        [ 0.02775339,  0.00940752, -0.02210233, ..., -0.01369841,
          0.02535246,  0.00544855],
        [ 0.02775339,  0.00940752, -0.02210233, ..., -0.01369841,
          0.02535246,  0.00544855],
        [ 0.02775339,  0.00940752, -0.02210233, ..., -0.01369841,
          0.02535246,  0.00544855]]], dtype=float32)>

In [48]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.00629101, 0.00173167, 0.        , 0.        , 0.        ,
         0.0435127 , 0.03820759, 0.07013372, 0.05012494, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.02892926, 0.00302002, 0.05109669, 0.06482902,
         0.        , 0.        , 0.02804936, 0.        , 0.06431925,
         0.05776182, 0.05861841, 0.07202369, 0.00463436, 0.        ,
         0.        , 0.09544349],
        [0.02641745, 0.        , 0.00067625, 0.03006024, 0.00116641,
         0.10010636, 0.02346494, 0.05440895, 0.06995148, 0.        ,
         0.        , 0.        , 0.        , 0.05630912, 0.02899412,
         0.00131629, 0.        , 0.        , 0.02363124, 0.        ,
         0.02152163, 0.        , 0.03682813, 0.        , 0.        ,
         0.03388753, 0.01818076, 0.00953974, 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.    

In [49]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.02641745, 0.05082145, 0.03244704, 0.03006024, 0.02641664,
        0.10010636, 0.03820759, 0.11416233, 0.06995148, 0.00814037,
        0.        , 0.01483488, 0.03608862, 0.05630912, 0.02899412,
        0.04217788, 0.03287478, 0.0507023 , 0.05109669, 0.06482902,
        0.02152163, 0.        , 0.04656328, 0.03323977, 0.06431925,
        0.05776182, 0.05861841, 0.10078599, 0.00834315, 0.04191744,
        0.01149672, 0.09544349]], dtype=float32)>

In [50]:
# Create 1D CNN layer to model sequences
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding="valid",
                           activation="relu")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5_history = model_5.fit(train_sentences,train_labels,
                              validation_data=(val_sentences,val_labels),
                              epochs=5)

Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.1530 - accuracy: 0.9457 - val_loss: 0.7502 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1020 - accuracy: 0.9626 - val_loss: 0.8878 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0802 - accuracy: 0.9691 - val_loss: 0.9619 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0693 - accuracy: 0.9730 - val_loss: 1.0411 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0605 - accuracy: 0.9749 - val_loss: 1.1020 - val_accuracy: 0.7690


In [51]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 969us/step


array([[2.1116717e-01],
       [6.1397952e-01],
       [9.9995577e-01],
       [4.7976971e-02],
       [2.9762123e-07],
       [9.9739838e-01],
       [9.9166274e-01],
       [9.9978369e-01],
       [9.9999893e-01],
       [7.4387318e-01]], dtype=float32)

In [52]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 76.9028871391076,
 'precision': 0.7697342660172047,
 'recall': 0.7690288713910761,
 'f1': 0.7674269881907728}

In [54]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 6 - Tensorflow Hub pretrained models
USE-extractor - universal sentence encoder (USE)

In [55]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "When you can the unvidersal sentence encider on a sentence, it turns into into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [56]:
embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157027,  0.0248591 ,  0.02878048, ..., -0.00186125,
         0.02315824, -0.01485021],
       [ 0.06090042, -0.08857404,  0.01838204, ..., -0.00653699,
         0.03106263,  0.0370101 ]], dtype=float32)>

Use the above USE"" embedding layer for a normal type of model

In [57]:
# Create a keras layer using the USE"" pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer(hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [58]:
sentence_encoder_layer

In [59]:
# Create  sequential layer
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6_history = model_6.fit(train_sentences,train_labels,
                               validation_data=(val_sentences,val_labels),
                               epochs=5)

Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.6501 - accuracy: 0.7362 - val_loss: 0.6150 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5824 - accuracy: 0.7901 - val_loss: 0.5650 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5389 - accuracy: 0.7933 - val_loss: 0.5329 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5100 - accuracy: 0.7990 - val_loss: 0.5115 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4897 - accuracy: 0.8013 - val_loss: 0.4969 - val_accuracy: 0.7874


In [60]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.35795432],
       [0.6764395 ],
       [0.85277396],
       [0.34947234],
       [0.64873725],
       [0.7345567 ],
       [0.8271819 ],
       [0.8393318 ],
       [0.7466447 ],
       [0.2045286 ]], dtype=float32)

In [61]:
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 78.74015748031496,
 'precision': 0.7878968015414602,
 'recall': 0.7874015748031497,
 'f1': 0.7862340407224946}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [64]:
# Challenge, beat the baseline
model_6_exp = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_6_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_6_exp_history = model_6_exp.fit(train_sentences,train_labels,
                                      epochs=5,
                                      validation_data=(val_sentences,val_labels))

model_6_exp_pred_probs = model_6_exp.predict(val_sentences)
print(model_6_exp_pred_probs[:10])

model_6_exp_preds = tf.squeeze(tf.round(model_6_exp_pred_probs))
print(model_6_exp_preds[:10])

model_6_exp_results = calculate_results(val_labels, model_6_exp_preds)
print(model_6_exp_results)

Epoch 1/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4888 - accuracy: 0.7767 - val_loss: 0.4380 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4022 - accuracy: 0.8218 - val_loss: 0.4231 - val_accuracy: 0.8202
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3812 - accuracy: 0.8378 - val_loss: 0.4229 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3612 - accuracy: 0.8475 - val_loss: 0.4174 - val_accuracy: 0.8189
Epoch 5/5
24/24 [==============================] - 0s 5ms/step
[[0.12460937]
 [0.7986058 ]
 [0.99816906]
 [0.1912126 ]
 [0.75334513]
 [0.8186079 ]
 [0.984877  ]
 [0.99297863]
 [0.9755053 ]
 [0.1012126 ]]
tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 82.41469816272966, 'precision': 0.8256511205330104, 'recall': 0.8241469816272966, 'f1': 0.8230145947665632}


### Model 7 : TF Hub Pretrained USE but with 10% of training data

In [65]:
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [66]:
# Check the number of targets in the dataset
train_10_percent["target"].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [67]:
train_df_shuffled["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [68]:
# model 7
model_7 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])

model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7_history = model_7.fit(train_sentences_10_percent, train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))


Epoch 1/5
24/24 [==============================] - 1s 21ms/step - loss: 0.6745 - accuracy: 0.6347 - val_loss: 0.6350 - val_accuracy: 0.7913
Epoch 2/5
24/24 [==============================] - 0s 10ms/step - loss: 0.5776 - accuracy: 0.8068 - val_loss: 0.4946 - val_accuracy: 0.8241
Epoch 3/5
24/24 [==============================] - 0s 10ms/step - loss: 0.4486 - accuracy: 0.8187 - val_loss: 0.3936 - val_accuracy: 0.8425
Epoch 4/5
24/24 [==============================] - 0s 10ms/step - loss: 0.3795 - accuracy: 0.8463 - val_loss: 0.3396 - val_accuracy: 0.8609
Epoch 5/5
24/24 [==============================] - 0s 10ms/step - loss: 0.3330 - accuracy: 0.8686 - val_loss: 0.2966 - val_accuracy: 0.8832


In [72]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 0s 6ms/step


array([[0.09911713],
       [0.8881445 ],
       [0.9659027 ],
       [0.36552706],
       [0.9254515 ],
       [0.9094441 ],
       [0.9597035 ],
       [0.98577917],
       [0.91345173],
       [0.01857546]], dtype=float32)

In [73]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [74]:
model_7_results = calculate_results(val_labels,model_7_preds)
model_7_results

{'accuracy': 88.32020997375328,
 'precision': 0.8836591055325493,
 'recall': 0.8832020997375328,
 'f1': 0.8828731455017578}

model 7 did very good compared to model 6, model is same, but only 10% data, why?
what could the reason be:
1. maybe the way we created 10% data

ANS : train_sentences_10_percent has little amount of val_sentences, becuase we took 10% from whole train_df_shuffled variable

So, it is already seeing what it will be validating on.

This is called data leak problem, some part of validation/testing data leaks into training data

In [77]:
# Making correct data allocation - without data leakage
train_10_percent_split = int(0.1 * len(train_sentences))    # train_sentences are already in random order, no need to shuffle
train_sentence_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_sentence_10_percent), len(train_labels_10_percent)

(685, 685)

In [79]:
model_7 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
mdoel_7_history_split = model_7.fit(train_sentence_10_percent, train_labels_10_percent,
                                    epochs=5,
                                    validation_data=(val_sentences,val_labels))

Epoch 1/5
22/22 [==============================] - 1s 24ms/step - loss: 0.6678 - accuracy: 0.5912 - val_loss: 0.6525 - val_accuracy: 0.5472
Epoch 2/5
22/22 [==============================] - 0s 13ms/step - loss: 0.5857 - accuracy: 0.6832 - val_loss: 0.5773 - val_accuracy: 0.7520
Epoch 3/5
22/22 [==============================] - 0s 12ms/step - loss: 0.4762 - accuracy: 0.8307 - val_loss: 0.5131 - val_accuracy: 0.7808
Epoch 4/5
22/22 [==============================] - 0s 12ms/step - loss: 0.3910 - accuracy: 0.8394 - val_loss: 0.5049 - val_accuracy: 0.7822
Epoch 5/5
22/22 [==============================] - 0s 12ms/step - loss: 0.3346 - accuracy: 0.8599 - val_loss: 0.5143 - val_accuracy: 0.7756


In [80]:
model_7_pred_probs = model_7.predict(val_sentences)
print(model_7_pred_probs[:10])

model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
print(model_7_preds[:10])

model_7_results = calculate_results(val_labels, model_7_preds)
print(model_7_results)

24/24 [==============================] - 0s 5ms/step
[[0.0608828 ]
 [0.5566165 ]
 [0.94135475]
 [0.42704904]
 [0.67850745]
 [0.6879722 ]
 [0.9303501 ]
 [0.859324  ]
 [0.92495406]
 [0.08896577]]
tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 77.55905511811024, 'precision': 0.7759863909628747, 'recall': 0.7755905511811023, 'f1': 0.7743062301518678}
